In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
#import xgboost as xgb
from sklearn.metrics import f1_score
#import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
#import lightgbm as lgbm
import scipy
from sklearn.preprocessing import StandardScaler
import cv2
from random import shuffle

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
%matplotlib inline
%pylab inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

/home/afattahov/tf3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


/home/afattahov/tf3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['shuffle']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K

# create the base pre-trained model
base_model = VGG19(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
#x = Dense(1024, activation='relu')(x)
x = Dropout(0.3)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

Using TensorFlow backend.
/home/afattahov/.local/lib/python3.5/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [4]:
import random
import cv2
import os
zir_files = os.listdir('data/zir')
norm_files = os.listdir('data/norm')

train_zir = zir_files[:55]
val_zir = zir_files[55:]

train_norm = norm_files[:14]
val_norm = norm_files[14:]

In [5]:
len(zir_files), len(norm_files)

(77, 22)

In [31]:
def all_generator(batch_size):
    while(1):
        #shuffle(train_zir)
        #shuffle(train_norm)
        
        x_train = []
        y_train = []
        for i in range(batch_size):
            zir = random.randint(0,1)
            if zir == 1:
                idx = random.randint(0,54)
                img = cv2.imread('zir/{}'.format(train_zir[idx]))
                y_train += [1]
            else:
                idx = random.randint(0,13)
                img = cv2.imread('norm/{}'.format(train_norm[idx]))
                y_train += [0]  
                
            x = random.randint(0,img.shape[0]-224)
            y = random.randint(0,img.shape[1]-224)

            img = img[x:x+224,y:y+224,:]
            
            #img = cv2.resize(img, (224, 224))
            num_rows, num_cols = img.shape[:2]

            rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), random.choice([0, 90, 180, 270]), 1)
            img = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))

            if random.choice([0, 1]) == 1:
                img = cv2.flip( img, 0  )

            #1
            x_train.append(img)
            #y_train.append(targets)  
            
            
        y_train = np.array(y_train, np.uint8)
        x_train = np.array(x_train, np.float16) / 255.0
        yield (x_train, y_train)

In [32]:
def val_generator(batch_size):
    while(1):
        #shuffle(val_zir)
        #shuffle(val_norm)
        
        x_train = []
        y_train = []
        for i in range(batch_size):
            zir = random.randint(0,1)
            if zir == 1:
                idx = random.randint(0,21)
                img = cv2.imread('zir/{}'.format(val_zir[idx]))
                y_train += [1]
            else:
                idx = random.randint(0,7)
                img = cv2.imread('norm/{}'.format(val_norm[idx]))
                y_train += [0]  
                
            x = random.randint(0,img.shape[0]-224)
            y = random.randint(0,img.shape[1]-224)

            img = img[x:x+224,y:y+224,:]
            
            #img = cv2.resize(img, (224, 224))
            num_rows, num_cols = img.shape[:2]

            rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), random.choice([0, 90, 180, 270]), 1)
            img = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))

            if random.choice([0, 1]) == 1:
                img = cv2.flip( img, 0  )

            #1
            x_train.append(img)
            #y_train.append(targets)  
            
            
        y_train = np.array(y_train, np.uint8)
        x_train = np.array(x_train, np.float16) / 255.0
        yield (x_train, y_train)

In [34]:
import keras
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

from keras.callbacks import Callback, ModelCheckpoint


# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer=adam,
              metrics=['accuracy'])

model_checkpoint = ModelCheckpoint('VGG16_.h5', save_best_only=True, save_weights_only=True)

In [35]:
batch_size = 64
model.fit_generator(all_generator(batch_size),
                    samples_per_epoch=10,
                    validation_data=val_generator(batch_size),
                    nb_val_samples=5,
                    nb_epoch=40,
                    verbose=1,
                    callbacks=[model_checkpoint])

/home/afattahov/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
  
/home/afattahov/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=40, validation_steps=5, callbacks=[<keras.ca..., steps_per_epoch=10, validation_data=<generator..., verbose=1)`
  


Epoch 1/40
10/10 [==============================] - 66s - loss: 0.8145 - acc: 0.4766 - val_loss: 0.7002 - val_acc: 0.4656
Epoch 2/40
10/10 [==============================] - 39s - loss: 0.6670 - acc: 0.5547 - val_loss: 0.5899 - val_acc: 0.7656
Epoch 3/40
10/10 [==============================] - 40s - loss: 0.5942 - acc: 0.7344 - val_loss: 0.5449 - val_acc: 0.9250
Epoch 4/40
10/10 [==============================] - 40s - loss: 0.5511 - acc: 0.8063 - val_loss: 0.4962 - val_acc: 0.9844
Epoch 5/40
10/10 [==============================] - 40s - loss: 0.5097 - acc: 0.8625 - val_loss: 0.4598 - val_acc: 0.9531
Epoch 6/40
10/10 [==============================] - 40s - loss: 0.4811 - acc: 0.8891 - val_loss: 0.4333 - val_acc: 0.9781
Epoch 7/40
10/10 [==============================] - 40s - loss: 0.4413 - acc: 0.9125 - val_loss: 0.4009 - val_acc: 0.9750
Epoch 8/40
10/10 [==============================] - 40s - loss: 0.4235 - acc: 0.9234 - val_loss: 0.3647 - val_acc: 0.9875
Epoch 9/40
10/10 [======

In [36]:
batch_size = 64
model.fit_generator(all_generator(batch_size),
                    samples_per_epoch=10,
                    validation_data=val_generator(batch_size),
                    nb_val_samples=5,
                    nb_epoch=40,
                    verbose=1,
                    callbacks=[model_checkpoint])

/home/afattahov/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
  
/home/afattahov/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=40, validation_steps=5, callbacks=[<keras.ca..., steps_per_epoch=10, validation_data=<generator..., verbose=1)`
  


Epoch 1/40
10/10 [==============================] - 64s - loss: 0.1481 - acc: 0.9703 - val_loss: 0.1172 - val_acc: 0.9812
Epoch 2/40
10/10 [==============================] - 40s - loss: 0.1313 - acc: 0.9812 - val_loss: 0.1047 - val_acc: 0.9906
Epoch 3/40
10/10 [==============================] - 40s - loss: 0.1411 - acc: 0.9719 - val_loss: 0.1108 - val_acc: 0.9844
Epoch 4/40
10/10 [==============================] - 39s - loss: 0.1450 - acc: 0.9734 - val_loss: 0.1090 - val_acc: 0.9938
Epoch 5/40
10/10 [==============================] - 41s - loss: 0.1385 - acc: 0.9656 - val_loss: 0.1054 - val_acc: 0.9938
Epoch 6/40
10/10 [==============================] - 40s - loss: 0.1373 - acc: 0.9734 - val_loss: 0.1012 - val_acc: 0.9969
Epoch 7/40
10/10 [==============================] - 42s - loss: 0.1445 - acc: 0.9688 - val_loss: 0.0949 - val_acc: 0.9938
Epoch 8/40
10/10 [==============================] - 40s - loss: 0.1246 - acc: 0.9672 - val_loss: 0.1037 - val_acc: 0.9906
Epoch 9/40
10/10 [======

In [37]:
for layer in model.layers:
    layer.trainable = False
    
import keras
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

from keras.callbacks import Callback, ModelCheckpoint


# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer=adam,
              metrics=['accuracy'])

model_checkpoint = ModelCheckpoint('VGG16.h5', save_best_only=True, save_weights_only=True)

In [38]:
batch_size = 64
model.fit_generator(all_generator(batch_size),
                    samples_per_epoch=10,
                    validation_data=val_generator(batch_size),
                    nb_val_samples=5,
                    nb_epoch=40,
                    verbose=1,
                    callbacks=[model_checkpoint])

/home/afattahov/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
  
/home/afattahov/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=40, validation_steps=5, callbacks=[<keras.ca..., steps_per_epoch=10, validation_data=<generator..., verbose=1)`
  


Epoch 1/40
10/10 [==============================] - 64s - loss: 0.0902 - acc: 0.9828 - val_loss: 0.0544 - val_acc: 1.0000
Epoch 2/40
10/10 [==============================] - 41s - loss: 0.0793 - acc: 0.9938 - val_loss: 0.0716 - val_acc: 0.9875
Epoch 3/40
10/10 [==============================] - 40s - loss: 0.0784 - acc: 0.9938 - val_loss: 0.0752 - val_acc: 0.9875
Epoch 4/40
10/10 [==============================] - 40s - loss: 0.0899 - acc: 0.9750 - val_loss: 0.0704 - val_acc: 0.9906
Epoch 5/40
10/10 [==============================] - 40s - loss: 0.0929 - acc: 0.9859 - val_loss: 0.0573 - val_acc: 0.9938
Epoch 6/40
10/10 [==============================] - 40s - loss: 0.0797 - acc: 0.9859 - val_loss: 0.0637 - val_acc: 0.9938
Epoch 7/40
10/10 [==============================] - 41s - loss: 0.1023 - acc: 0.9719 - val_loss: 0.0523 - val_acc: 1.0000
Epoch 8/40
10/10 [==============================] - 41s - loss: 0.0935 - acc: 0.9766 - val_loss: 0.0470 - val_acc: 1.0000
Epoch 9/40
10/10 [======